In [79]:
import pandas as pd
import folium
import json
import numpy as np
import requests

In [43]:
df_map = pd.read_csv('./station_coordinate.csv', encoding='UTF-8')

In [44]:
df_map

,line,name,code,lat,lng
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382
...,...,...,...,...,...
725,인천선,지식정보단지,3135.0,37.378384,126.645168
726,인천선,인천대입구,3136.0,37.386007,126.639484
727,인천선,센트럴파크,3137.0,37.393054,126.634729
728,인천선,국제업무지구,3138.0,37.399907,126.630347


In [45]:
#  01~09호선 분리
df_map=df_map.iloc[:439,]
df_map_1=df_map[df_map['line']=='01호선']
df_map_2=df_map[df_map['line']=='02호선']
df_map_3=df_map[df_map['line']=='03호선']
df_map_4=df_map[df_map['line']=='04호선']
df_map_5=df_map[df_map['line']=='05호선']
df_map_5=df_map_5.dropna(axis=0) # 누락데이터가 서울이 아니라 삭제
df_map_6=df_map[df_map['line']=='06호선']
df_map_7=df_map[df_map['line']=='07호선']
df_map_8=df_map[df_map['line']=='08호선']
df_map_9=df_map[df_map['line']=='09호선']

In [76]:
# 누락 데이터 수정
a=df_map_9[df_map_9['name']=='석촌고분'].index
df_map_9.loc[a, 'lat'] = 37.502453
df_map_9.loc[a,'lng']=127.096961
b=df_map_9[df_map_9['name']=='송파나루'].index
df_map_9.loc[b, 'lat'] = 37.510303
df_map_9.loc[b,'lng']=127.112231
c=df_map_9[df_map_9['name']=='한성백제'].index
df_map_9.loc[c, 'lat'] = 37.516311
df_map_9.loc[c,'lng']=127.116168
d=df_map_9[df_map_9['name']=='둔촌오륜'].index
df_map_9.loc[d, 'lat'] = 37.519395
df_map_9.loc[d,'lng']=127.138706
e=df_map_9[df_map_9['name']=='삼전'].index
df_map_9.loc[e, 'lat'] = 37.504356
df_map_9.loc[e,'lng']=127.08804
f=df_map_9[df_map_9['name']=='중앙보훈병원'].index
df_map_9.loc[f, 'lat'] = 37.528321
df_map_9.loc[f,'lng']=127.148376

In [100]:
map=folium.Map(location=[df_map.lat.mean(),df_map.lng.mean()], zoom_start=12,
               tiles='Stamen Terrain') 

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

for i in df_map_1.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_1.lat[i],df_map_1.lng[i]],
        popup=folium.Popup(df_map_1.line[i], max_width=200),
        tooltip=df_map_1.name[i],
        color='darkblue',
        fill=True,
        fill_color='darkblue'
    ).add_to(map)

for i in df_map_2.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_2.lat[i],df_map_2.lng[i]],
        popup=folium.Popup(df_map_2.line[i], max_width=200),
        tooltip=df_map_2.name[i],
        color='darkgreen',
        fill=True,
        fill_color='darkgreen'
    ).add_to(map)

for i in df_map_3.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_3.lat[i],df_map_3.lng[i]],
        popup=folium.Popup(df_map_3.line[i], max_width=200),
        tooltip=df_map_3.name[i],
        color='darkorange',
        fill=True,
        fill_color='darkorange'
    ).add_to(map)
    
for i in df_map_4.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_4.lat[i],df_map_4.lng[i]],
        popup=folium.Popup(df_map_4.line[i], max_width=200),
        tooltip=df_map_4.name[i],
        color='darkturquoise',
        fill=True,
        fill_color='darkturquoise'
    ).add_to(map)
    
for i in df_map_5.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_5.lat[i],df_map_5.lng[i]],
        popup=folium.Popup(df_map_5.line[i], max_width=200),
        tooltip=df_map_5.name[i],
        color='slateblue',
        fill=True,
        fill_color='slateblue'
    ).add_to(map)
    
for i in df_map_6.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_6.lat[i],df_map_6.lng[i]],
        popup=folium.Popup(df_map_6.line[i], max_width=200),
        tooltip=df_map_6.name[i],
        color='chocolate',
        fill=True,
        fill_color='chocolate'
    ).add_to(map)
    
for i in df_map_7.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_7.lat[i],df_map_7.lng[i]],
        popup=folium.Popup(df_map_7.line[i], max_width=200),
        tooltip=df_map_7.name[i],
        color='olive',
        fill=True,
        fill_color='olive'
    ).add_to(map)
    
for i in df_map_8.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_8.lat[i],df_map_8.lng[i]],
        popup=folium.Popup(df_map_8.line[i], max_width=200),
        tooltip=df_map_8.name[i],
        color='dippink',
        fill=True,
        fill_color='dippink'
    ).add_to(map)
    
for i in df_map_9.index:
    folium.Circle(
        radius=200, #지름
        location=[df_map_9.lat[i],df_map_9.lng[i]],
        popup=folium.Popup(df_map_9.line[i], max_width=200),
        tooltip=df_map_9.name[i],
        color='tan',
        fill=True,
        fill_color='tan'
    ).add_to(map)
    
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

#folium.GeoJson(seoul_geo).add_to(map)
#folium.PolyLine(locations = df_map_1.iloc[:,3:]).add_to(map)
title='<h3 align="center" style="font-size:20px">서울시 지하철 노선</h3>'
map.get_root().html.add_child(folium.Element(title))

map